# Hypothesis testing Solutions

### **Exercise 1**

#### ANOVA

Suppose that a study wants to check if there is a significant difference between the goal averages of soccer players depending on the position in which they play. In case there is a difference, you want to know which positions differ from the rest.

NOTE: You must replace the values <<<FIXME>>>.

**Exercise: Load data from "datos_laliga.csv". It contains a sample of randomly selected players.**

In [25]:
import pandas as pd
my_data = pd.read_csv("datos_laliga.csv")
my_data

AttributeError: module 'numpy' has no attribute 'ndarray'

In [23]:
my_data.describe()

NameError: name 'my_data' is not defined

**Exercise: Identify the number of groups and number of observations per group to determine if it is a balanced model. The mean and standard deviation of the group are also calculated.**

In [20]:
pd.crosstab(my_data["position"],columns=["DC", "MO", "MP", "P"])
#DC: Delantero centro
#MO: Medio ofensivo
#MP: Media punta
#P: Puntero

NameError: name 'pd' is not defined

**Exercise: Calculate the mean by position**

In [ ]:
my_data.groupby('position')['average'].agg('mean')

**Exercise: Calculate the standard deviation by position**

In [ ]:

my_data.groupby('position')['average'].agg('std')

Since the number of observations per group is not constant, it is an unbalanced model. It is important to take this into account when checking the conditions of normality and homoscedasticity (s1 = s2 = s3 = s4). The most useful graphical representation before performing an ANOVA is the Box-Plot model.

**Exercise: Plot a boxplot for each position**

In [19]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(data=my_data, x="position", y="average")

plt.xlabel("Position")
plt.ylabel("Average")
plt.title("Boxplot of Average by Position")

plt.show()

AttributeError: module 'numpy' has no attribute 'ndarray'

#### Independence

The total sample size is <10% of the population of all players in the league. The groups (categorical variable) are independent of each other since a random sample of players from the entire league (not just from the same team) has been made.

Normal distribution of observations: The quantitative variable must be distributed in a normal way in each of the groups. The normality study can be done graphically (qqplot) or with a hypothesis test.

**Exercise: Make an analysis about normal distribution for each position**

In [ ]:
import numpy as np 
import pylab 
import matplotlib.pyplot as plt
import scipy.stats as stats

measurements = my_data.loc[my_data["position"] == "P","average"]
stats.probplot(measurements, dist="norm", plot=pylab)
plt.title("P")
plt.show()

measurements = my_data.loc[my_data["position"] == "MO","average"]
stats.probplot(measurements, dist="norm", plot=pylab)
plt.title("MO")
plt.show()

measurements = my_data.loc[my_data["position"] == "DC","average"]
stats.probplot(measurements, dist="norm", plot=pylab)
plt.title("DC")
plt.show()

measurements = my_data.loc[my_data["position"] == "MP","average"]
stats.probplot(measurements, dist="norm", plot=pylab)
plt.title("MP")
plt.show()

**Exercise: make the boxplot for each position, what you can say about them?**

In [ ]:
# Using plotly
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Box(y=my_data.loc[my_data["position"] == "P","average"]))
fig.add_trace(go.Box(y=my_data.loc[my_data["position"] == "MO","average"]))
fig.add_trace(go.Box(y=my_data.loc[my_data["position"] == "DC","average"]))
fig.add_trace(go.Box(y=my_data.loc[my_data["position"] == "MP","average"]))
fig.show()

**Exercise: Use the Kolmogorov-Smirnov test or with or without the Lilliefors correction in order to know the normality distribution.**

In [ ]:
# Beginner way to do it

from statsmodels.stats.diagnostic import lilliefors

my_df = pd.DataFrame(index=np.arange(len(np.unique(my_data["position"]))), columns=["position", "D_statistic", "p_value"])
my_df["position"] = np.unique(my_data["position"])

for position in my_df["position"]:
    my_data_subset = my_data.loc[my_data["position"] == position,:]
    D_statistic, p_value = lilliefors(my_data_subset.average)
    my_df.loc[my_df["position"]==position,["D_statistic", "p_value"]] = D_statistic, p_value
    
print(my_df)

In [ ]:
#Another way to do it (Highly recommendable)
my_data.groupby("position")["average"].apply(lilliefors)

The hypothesis tests do not show evidence of a lack of normality.

Constant variance between groups (homoscedasticity):

Given that there is a group (DC) that is at the limit to accept that it is distributed in a normal way, the Fisher and Bartlett tests are not recommended. Instead it is better to use a test based on the median Levene test or the Fligner-Killeen test.

**Exercise: use the fligner and levene functions from scipy.stats in order to know the homocedasticy**

In [ ]:
from scipy import stats

values_array = pd.DataFrame(my_data.groupby("position")["average"]).to_numpy()

print(stats.fligner(values_array[0,1], values_array[1,1], values_array[2,1], values_array[3,1]))

print(stats.levene(values_array[0,1], values_array[1,1], values_array[2,1], values_array[3,1]))

# stats.fligner(values_array[:,1]) # It doesn't work, please analyse

There is no significant evidence of lack of homoscedasticity in either of the two tests.

The study of the conditions can be carried out after calculating the ANOVA, since if they are not fulfilled, it does not make much sense to continue. However, the most appropriate way to verify that the necessary conditions are satisfied is by studying the model residuals once the ANOVA has been generated.

**Exercise: make an ANOVA table and analyze the p-value using the packages statsmodels and the bioinfokit**

In [ ]:
# get ANOVA table as R like output
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Ordinary Least Squares (OLS) model
model = ols('average ~ position', data=my_data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

In [ ]:
# ANOVA with package bioinfokit
from bioinfokit.analys import stat
res = stat()
res.anova_stat(df=my_data, res_var='value', anova_model='average ~ position')
res.anova_summary
# output (ANOVA F and p value)

**Exercise: make a plot of the fitted values vs residuals. Make the plot of the Standardized Residuals. Make the histogram of the residuals.**

In [ ]:
# QQ-plot
import statsmodels.api as sm
import matplotlib.pyplot as plt


plt.scatter(res.anova_model_out.fittedvalues, res.anova_model_out.resid)
plt.xlabel("Fitted values")
plt.ylabel("Residuals")
plt.title("Residuals vs. Fitted")
plt.show()

# res.anova_std_residuals are standardized residuals obtained from ANOVA (check above)
sm.qqplot(res.anova_std_residuals, line='45')
plt.xlabel("Theoretical Quantiles")
plt.ylabel("Standardized Residuals")
plt.show()

# histogram
plt.hist(res.anova_model_out.resid, bins='auto', histtype='bar', ec='k') 
plt.xlabel("Residuals")
plt.ylabel('Frequency')
plt.show()

Given that the p-value is higher than 0.05, there is not enough evidence to consider that at least two means are different. The graphical representation of the residuals does not show lack of homoscedasticity (graph 1) and in the qqplot the residuals are distributed very close to the normal line (graph 2 and 3).

### **Exercise 2**

### T-TEST INDEPENDENT

A university professor gave online lectures instead of face-to-face classes due to Covid-19. Later, he uploaded recorded lectures to the cloud for students who followed the course asynchronously (those who did not attend the lesson but later watched the records). However, he believes that the students who attend class at the class time and participate in the process are more successful. Therefore, he recorded the average grades of the students at the end of the semester. The data is below.

```py
synchronous = [94. , 84.9, 82.6, 69.5, 80.1, 79.6, 81.4, 77.8, 81.7, 78.8, 73.2, 87.9, 87.9, 93.5, 82.3, 79.3, 78.3, 71.6, 88.6, 74.6, 74.1, 80.6]

asynchronous = [77.1, 71.7, 91. , 72.2, 74.8, 85.1, 67.6, 69.9, 75.3, 71.7, 65.7, 72.6, 71.5, 78.2]
```

Conduct the hypothesis testing to check whether the professor’s belief is statistically significant by using a 0.05 significance level to evaluate the null and alternative hypotheses. Before doing hypothesis testing, check the related assumptions. Comment on the results.

In [5]:
import numpy as np
from scipy import stats
import pandas as pd

In [6]:
def check_normality(data):
    test_stat_normality, p_value_normality=stats.shapiro(data)
    print("p value:%.4f" % p_value_normality)
    if p_value_normality <0.05:
        print("Reject null hypothesis >> The data is not normally distributed")
    else:
        print("Fail to reject null hypothesis >> The data is normally distributed")       

In [7]:
def check_variance_homogeneity(group1, group2):
    test_stat_var, p_value_var= stats.levene(group1,group2)
    print("p value:%.4f" % p_value_var)
    if p_value_var <0.05:
        print("Reject null hypothesis >> The variances of the samples are different.")
    else:
        print("Fail to reject null hypothesis >> The variances of the samples are same.")


### Solution

In [8]:
sync = np.array([94. , 84.9, 82.6, 69.5, 80.1, 79.6, 81.4, 77.8, 81.7, 78.8, 73.2,
       87.9, 87.9, 93.5, 82.3, 79.3, 78.3, 71.6, 88.6, 74.6, 74.1, 80.6])
asyncr = np.array([77.1, 71.7, 91. , 72.2, 74.8, 85.1, 67.6, 69.9, 75.3, 71.7, 65.7, 72.6, 71.5, 78.2])

H₀: The data is normally distributed.

H₁: The data is not normally distributed.

In [9]:

check_normality(sync)
check_normality(asyncr)

p value:0.6556
Fail to reject null hypothesis >> The data is normally distributed
p value:0.0803
Fail to reject null hypothesis >> The data is normally distributed


H₀: The variances of the samples are the same.

H₁: The variances of the samples are different.

In [10]:

check_variance_homogeneity(sync, asyncr)

p value:0.8149
Fail to reject null hypothesis >> The variances of the samples are same.


In [11]:
ttest,p_value = stats.ttest_ind(sync,asyncr)
print("p value:%.8f" % p_value)
print("since the hypothesis is one sided >> use p_value/2 >> p_value_one_sided:%.4f" %(p_value/2))
if p_value/2 <0.05:
    print("Reject null hypothesis")
else:
    print("Fail to reject null hypothesis")

p value:0.00753598
since the hypothesis is one sided >> use p_value/2 >> p_value_one_sided:0.0038
Reject null hypothesis


At this significance level, there is enough evidence to conclude that the average grade of the students who follow the course synchronously is higher than the students who follow the course asynchronously.

Source:

https://towardsdatascience.com/hypothesis-testing-with-python-step-by-step-hands-on-tutorial-with-practical-examples-e805975ea96e